# Inference

## Part 1

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd

from torch.nn import BCEWithLogitsLoss
loss_fn = BCEWithLogitsLoss()

In [1]:

# Load saved model
save_directory = "./model_part1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model = BertForSequenceClassification.from_pretrained(save_directory)
model.to(device)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(save_directory)

# Inference function
def run_inference(model, tokenizer, text, device):
    """
    Runs inference using the pretrained model and tokenizer.
    
    Parameters:
        model: The pretrained model
        tokenizer: The tokenizer
        text: The text to run inference on
        device: The device type ('cpu' or 'cuda')
    
    Returns:
        prediction: 0 or 1 (based on the logit)
    """
    # Tokenize the text
    input_data = tokenizer([text], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
    
    # Load tensor to the specified device
    input_ids = input_data['input_ids'].to(device)
    attention_mask = input_data['attention_mask'].to(device)
    token_type_ids = input_data['token_type_ids'].to(device)
    
    # Put model in evaluation mode
    model.eval()
    
    # Run inference
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    
    # Get logits
    logits = outputs.logits.squeeze().item()
    
    # Convert logits to predictions
    prediction = int(logits > 0)  # This line is modified

    return prediction

# Run inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

text = "Student Age: 7 Student Grade: 3 Source: mocktest Passage Name: dorian gray Genre: fiction Language Complexity: Extremely Difficult Sentence Structure: Extremely Difficult Themes and Content: Extremely Difficult Overall Complexity: Extremely Difficult Question: In whose residence does this passage take place? Question Difficulty: Challenging Question Type: Factual recall: there is a sentence that basically tells them the answer directly Answer Option A: Dorian Gray’s Answer Option B: Alan Campbell’s Answer Option C: Lord Henry Wotton’s Answer Option D: No one’s, it is occurring in public. Answer Option E: It is not mentioned in the text. Correct Answer: A Answer By Student: A Hour of exam: 14 Attention Level: High "

prediction = run_inference(model, tokenizer, text, device)

print(f"Prediction: {prediction}")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: c:\Python311\Lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary c:\Python311\Lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...
Prediction: 1


## Part 2

In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import pickle 
from torch.nn import CrossEntropyLoss

In [3]:


# Load
with open("./model_part2_v2/label_encoder.pkl", "rb") as f:
    encoder = pickle.load(f)
    
# Directory where model will be saved
save_directory = "./model_part2_v2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model = BertForSequenceClassification.from_pretrained(save_directory)
model.to(device)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(save_directory)

def run_multiclass_inference(model, tokenizer, encoder, text, device):
    """
    Runs inference using a pretrained model and tokenizer for multi-class classification.
    
    Parameters:
        model: The pretrained model
        tokenizer: The tokenizer
        text: The text to run inference on
        device: The device type ('cpu' or 'cuda')
    
    Returns:
        prediction: Label index with the highest logit value
    """
    # Tokenize the text
    input_data = tokenizer([text], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
    
    # Load tensor to the specified device
    input_ids = input_data['input_ids'].to(device)
    attention_mask = input_data['attention_mask'].to(device)
    token_type_ids = input_data['token_type_ids'].to(device)
    
    # Put model in evaluation mode
    model.eval()
    
    # Run inference
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    
    # Get logits
    logits = outputs.logits
    
    # Get label index with the maximum logit
    prediction = torch.argmax(logits, dim=1).cpu().item()

    return prediction

text = "Student Age: 7 Student Grade: 3 Source: mocktest Passage Name: dorian gray Genre: fiction Language Complexity: Extremely Difficult Sentence Structure: Extremely Difficult Themes and Content: Extremely Difficult Overall Complexity: Extremely Difficult Question: In whose residence does this passage take place? Question Difficulty: Challenging Question Type: Factual recall: there is a sentence that basically tells them the answer directly Answer Option A: Dorian Gray’s Answer Option B: Alan Campbell’s Answer Option C: Lord Henry Wotton’s Answer Option D: No one’s, it is occurring in public. Answer Option E: It is not mentioned in the text. Correct Answer: A Answer By Student: A Hour of exam: 14 Attention Level: High "
text = "Student Age: 7 Student Grade: 3 Source: mocktest Passage Name: dorian gray Genre: fiction Language Complexity: Extremely Difficult Sentence Structure: Extremely Difficult Themes and Content: Extremely Difficult Overall Complexity: Extremely Difficult Question: ‘There was a look of contempt in the steady searching gaze that he turned on Dorian.’ (lines 2-3) Which of the following best describes Alan’s initial assessment of Dorian? Question Difficulty: Difficult Question Type: Factual recall: there is a sentence that basically tells them the answer directly Answer Option A: He is not familiar with him and is thus curious. Answer Option B: He respects him but isn’t particularly interested. Answer Option C: He is wary of him and looking out for danger. Answer Option D: He detests him and wants to turn away from him. Answer Option E: He is scornful and evaluating. Correct Answer: E Answer By Student: D Hour of exam: 14 Attention Level: High "

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
prediction_index = run_multiclass_inference(model, tokenizer, encoder, text, device)

decoded_prediction = encoder.inverse_transform([prediction_index])

print(f"Prediction Index: {prediction_index}")
print(f"Decoded Prediction: {decoded_prediction[0]}")


Prediction Index: 3
Decoded Prediction: D
